<font size=8>  Setting up the Collection Space Navigator

In this How-To guide you will produce all necessary files to create a custom version of the Collection Space Navigator (CSN).

Project link: https://collection-space-navigator.github.io/ 

> Note: We highly recommend to first get familiar with the example collection before trying your with your own data.

<font size=6> 1) Prepare Collection Data

Loads the dataset and sets up the metadata.



In [ ]:
#@title (optional) Mount Google Drive
#@markdown >Note: This is not necessary if you work with example data. Mounting Google Drive gives your Colab Notebook instance access to your data, and does not share access with the CSN authors.

import ipywidgets as widgets
from ipywidgets import interactive,HBox,VBox,Label
from IPython.display import display
def mount_gdrive(v):
    try:
      from google.colab import drive
      # drive.mount(drive_path,force_remount=False)
      buttonGDrive.description="mounting..."
      buttonGDrive.disabled=True
      drive.mount('/content/gdrive',force_remount=True)
      %cd '/content/gdrive'
    except:
      print("...error mounting drive")
      buttonGDrive.description="mounting failed!"
    else:
      buttonGDrive.description="successfully mounted"

layoutButtons = {'width': '210px'}
buttonGDrive = widgets.Button(description='mount Google Drive',icon='check',indent=True,layout=layoutButtons)
buttonGDrive.on_click(mount_gdrive)

display(buttonGDrive)


In [ ]:
#@title Import Libraries
#@markdown Downloads the offical CSN repository from https://github.com/Collection-Space-Navigator/CSN and loads all necessary libraries.
#@markdown >Note: Optional libraries will be installed only if needed.

!git clone https://github.com/Collection-Space-Navigator/CSN
%cd CSN

import json, math, os, io
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interactive,HBox,VBox,Label
from IPython.display import display

In [ ]:
#@title Define INPUT
#@markdown >Note: Running this cell opens a dialog in which the input data can be defined. Use the provided example data (recommended first) or your own.

style = {'description_width': '250px'}
layout = {'width': '600px', 'justify-content': 'lex-satrt'}
useExample = widgets.Checkbox(value=True,description='use example data',indent=True)
datasetTitle = widgets.Text(placeholder='title of the dataset', description='Title:', style=style, layout=layout, value = "Testset")
description = widgets.Textarea(placeholder='Short description of the dataset and method(s)', description='Description (optional):', style=style, layout=layout, value="")
embeddingsLocation = widgets.Text(placeholder='path to embeddings file (.csv)', description='Embeddings Filepath (optional):', style=style, layout=layout, value = "CSN/example_data/embeddings_testset.csv")
metadataLocation = widgets.Text(placeholder='path to metadata file (.csv)', description='Metadata Filepath:', style=style, layout=layout, value = "CSN/example_data/metadata_testset.csv")
imageLocation = widgets.Text(placeholder='path to image collection folder', description='Image Folder:', style=style, layout=layout, value = "CSN/example_data/testset_images/")
imageWebLocation = widgets.Text(placeholder='URL prefix to public image directory', description='Image URL prefix:',style=style, layout=layout, value =  "https://github.com/Collection-Space-Navigator/CSN/raw/main/example_data/testset_images/")

if os.path.exists("/content/gdrive/MyDrive/"):
    imageLocation.value = "/content/gdrive/MyDrive/CSN/example_data/testset_images/"
    embeddingsLocation.value = "/content/gdrive/MyDrive/CSN/example_data/embeddings_testset.csv"
    metadataLocation.value = "/content/gdrive/MyDrive/CSN/example_data/metadata_testset.csv"  
else:
    imageLocation.value = "example_data/testset_images/"
    embeddingsLocation.value = "example_data/embeddings_testset.csv"
    metadataLocation.value = "example_data/metadata_testset.csv"  

imageWebLocation.value =  "https://github.com/Collection-Space-Navigator/CSN/raw/main/example_data/testset_images/"


subset = widgets.Checkbox(value=False,description='make subset',indent=True)
subsetSize = widgets.BoundedIntText(value=2048,min=10,max=9999999, step=1,description='Subset size:')
def makeSubset(SUBSET):
    if SUBSET:
        display(subsetSize)
    else:
        subsetSize.value == None
i = interactive(makeSubset, SUBSET = subset)
left = VBox([datasetTitle, description, embeddingsLocation, metadataLocation, imageLocation, imageWebLocation])
right = VBox([useExample, i])
display(HBox([left,right]))

In [ ]:
#@title Load INPUT files
#@markdown Loads and checks all files.


imagNumb = len(os.listdir(imageLocation.value))
print(f'found {imagNumb} files in {imageLocation.value}')
mappings = []

metadata = pd.read_csv(metadataLocation.value, skipinitialspace=True)
if subset.value:
    metadata = metadata[:subsetSize.value]
metaNumb = len(metadata)
print(f'found {metaNumb} entries in {metadataLocation.value}')

if embeddingsLocation.value != "":
  embeddings = pd.read_csv(embeddingsLocation.value, skipinitialspace=True)
  embeddings = embeddings.loc[:, embeddings.columns!='id']
  embeddings = embeddings.loc[:, embeddings.columns!='ID']
  if subset.value:
    embeddings = embeddings[:subsetSize.value]
  vecNumb = len(embeddings)
  print(f'found {vecNumb} entries in {embeddingsLocation.value}')

  if metaNumb == vecNumb:
    if vecNumb <= imagNumb:
      print("Looks ok.")
      print()
      print(f'Embedding file contains {vecNumb} vectors in {len(embeddings.columns)} dimensions.')
      print("Metadata Head:")
      print(metadata.head())
    else:
      print()
      print("ERROR: number of images is smaller than number of vectors")

if metaNumb <= imagNumb:
  print("Looks ok.")
  print("Metadata Head:")
  print(metadata.head())
else:
  print()
  print("ERROR: number of images and metadata elements don't match!")
foldername = datasetTitle.value.lower().replace(" ","_")
print()
print(f'Creating new dataset directory: build/datasets/{foldername}...')
if not os.path.exists(f"build/datasets/{foldername}"):
    os.makedirs(f"build/datasets/{foldername}")
    print("... success")
else:
    print("... folder already exists (might overwrite existing files)")

In [ ]:
#@title Assign metadata fields
#@markdown Choose which field names in the metadata file should be used.   
#@markdown >Note: Select multiple values using shift+ctrl+mouseclick, shift+command+mouseclick, or shift+arrow keys.

filenameColumn = widgets.Dropdown(description="Image filenames (JPG or PNG):",options=[mf for mf in metadata.columns if pd.api.types.is_string_dtype(metadata[mf]) and metadata[mf].str.endswith((".jpg",".JPEG","JPG",".jpeg",".png",".PNG")).all()], style=style, layout=layout)
classColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if mf != "index"],description='optional: Cluster data:', style=style, layout=layout)
infoColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if mf != "index"],description='Info fields (display in preview):', style=style, layout=layout)
sliderColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"],description='Slider data (floats or integers):', style=style, layout=layout)
filterColumns = widgets.SelectMultiple(options=[mf for mf in metadata.columns if pd.api.types.is_string_dtype(metadata[mf]) and mf != 'URL'],description='optional: Filter & Search fields (string):', style=style, layout=layout)
if useExample.value == True:
  infoColumns.value = ("Prompt", "Colors", "Contrast", "File Size")
  sliderColumns.value = ("Colorfulness", "Colors", "Contrast", "File Size")
  filterColumns.value = ("Prompt",)
  classColumns.value = ("Class",)
left = VBox([filenameColumn, infoColumns, sliderColumns])
right = VBox([filterColumns, classColumns])
display(HBox([left,right]))

In [ ]:
# rename filename column
metadata.rename(columns={filenameColumn.value: "filename"}, inplace=True)

----------------

<font size=6> 2) Prepare Image Data

To handle large amounts of images efficiently, the CSN uses sprite sheets with multiple thumbnails behind the scenes. These sprite-sheets need to be generated.

In [ ]:
#@title Generate sprite sheets
#@markdown >Note: only needed for new datasets or to update existing tiles (skip this part if you already generated them)

from CSN_utils import ImageSpriteGenerator

sprite_generator = ImageSpriteGenerator(foldername, spriteSize=2048, spriteRows=32, imageFolder=imageLocation.value, files=metadata[filenameColumn.value]).generate()

<font size=6> 2b) Feature Extraction (optional)
 
Run simple image feature extraction method using a pretrained Convolutionial Neural Network (CNN).
>Note: This can take a while depending on your hardware setup and size of image dataset...

In [ ]:
#@title Import libraries

try: 
    from tensorflow.keras import applications, models, Model
    from tensorflow.keras.applications.resnet50 import preprocess_input
    from tensorflow.keras.preprocessing import image
except:
    print("Installing tensorflow via Pip")
    %pip install flask tensorflow
    from tensorflow.keras import applications, models, Model
    from tensorflow.keras.applications.resnet50 import preprocess_input
    from tensorflow.keras.preprocessing import image
try:
    from PIL import ImageFile
except:
    print("Installing Pillow via Pip")
    %pip install pillow

from feature_extraction import extract_features

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#@title Perform feature extraction 
#@markdown Runs the extraction method with the chosen model and saves the embeddings as a CSV file
#@markdown >Note: This will overwrite the Embeddings Filepath value from earlier (see 1. 'Define Input')
#@markdown Learn more about the pretrained models: https://keras.io/api/applications/
model = 'ResNet50' #@param {type:"string"}
#@markdown options: ResNet50, Xception, VGG16, VGG19, InceptionV3, MobileNet

files = [f'{imageLocation.value}/{file}' for file in metadata['filename']]
features = extract_features(files, model=model)
df = pd.DataFrame(features[0])
embeddings = df # overwrites the Input

# # save embeddings in csv file
# df_2 = pd.DataFrame(features[0])
# df_2["id"] = features[1]
# em_filename = f"subset_embeddings_{model}.csv"
# df_2.to_csv(em_filename, index=False)

----------------

<font size=6> 3) Generate Mappings

Mappings are plots containing 2D coordinates (x,y) of the image objects. 

Here are several methods you can run. The Collection Space Navigator can handle many mappings but needs at least one to work.

<font size=5> 3.1 From metadata (optional)

In [ ]:
#@title (optional) Create 2D plots
#@markdown Choose 2 metadadata fields (float or integer) and click "make plot". Repeat for every combination you want to add.
#@markdown >Note: Running this step opens a dialog in which you can chose X and Y dimensions from the available data and create additional 2D plots

from CSN_utils import SimplePlot

def plot(v):
    result = SimplePlot(foldername,A=AColumn.value, B=BColumn.value, metadata=metadata)
    filename = (AColumn.value + "_" + BColumn.value).replace(" ","")
    mappings.append({"name": filename, "file": f"{filename}.json"})
    
AColumn = widgets.Dropdown(description="x-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
BColumn = widgets.Dropdown(description="y-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
button2DPlot = widgets.Button(description='make plot',icon='check')
button2DPlot.on_click(plot)
left = VBox([AColumn,BColumn])
right = VBox([button2DPlot])
HBox([left,right])

<font size=5> 3.2 From embeddings (optional)

In [ ]:
#@title (optional) Run Principal Component Analysis (PCA)
components = 2 #@param {type:"number"}
add_slider = True #@param {type:"boolean"}
#@markdown >See PCA documentation: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.eA.html


from CSN_utils import PCAGenerator

PCAEembedding = PCAGenerator(foldername, scale=True, data=embeddings.values, components=components).generate()
mappings.append({"name": "PCA", "file": "PCA.json"})

# add columns to metadata for each component
for i in range(components):
  metadata[f"PC{i+1}"] = PCAEembedding[:,i]
  print(f"... added PC{i+1} to metadata")

# add slider for each component
if add_slider:
  sliderCols = list(sliderColumns.value)
  for i in range(components):
    sliderCols.append(f"PC{i+1}")

In [ ]:
#@title (optional) Run UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction
n_neighbors=15 #@param {type:"number"}
min_dist=0.18 #@param {type:"number"}
metric="correlation" #@param {type:"string"}
verbose=True #@param {type:"boolean"}
#@markdown >See UMAP documentation: https://umap-learn.readthedocs.io/en/latest/

try:
    from CSN_utils import UMAPGenerator
except:
    print("Installing UMAP via Pip")
    %pip install umap-learn
    from CSN_utils import UMAPGenerator

fullEmbeddings = UMAPGenerator(foldername, data=embeddings.values, n_neighbors=n_neighbors, min_dist=min_dist, metric=metric, verbose=verbose).generate()
mappings.append({"name": "UMAP", "file": "UMAP.json"})


In [ ]:
#@title (optional) Run t-SNE: t-distributed Stochastic Neighbor Embedding
n_components = 2 #@param {type:"number"}
verbose = 1 #@param {type:"number"}
random_state = 123 #@param {type:"number"}
#@markdown >See t-SNE documentation: https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

from CSN_utils import TSNEGenerator

tsneEembedding = TSNEGenerator(foldername, data=embeddings.values, n_components=n_components, verbose=verbose, random_state=random_state).generate()
mappings.append({"name": "t-SNE", "file": "TSNE.json"})


In [ ]:
#@title (optional) Create 2D plots
#@markdown Choose 2 metadadata fields (float or integer) and click "make plot". Repeat for every combination you want to add.
#@markdown >Note: Running this step opens a dialog in which you can chose X and Y dimensions from the available data and create additional 2D plots

from CSN_utils import SimplePlot

def plot(v):
    result = SimplePlot(foldername,A=AColumn.value, B=BColumn.value, metadata=metadata)
    filename = (AColumn.value + "_" + BColumn.value).replace(" ","")
    mappings.append({"name": filename, "file": f"{filename}.json"})
    
AColumn = widgets.Dropdown(description="x-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
BColumn = widgets.Dropdown(description="y-axis:",options=[mf for mf in metadata.columns if pd.api.types.is_numeric_dtype(metadata[mf]) and mf != "index"], style=style, layout=layout)
button2DPlot = widgets.Button(description='make plot',icon='check')
button2DPlot.on_click(plot)
left = VBox([AColumn,BColumn])
right = VBox([button2DPlot])
HBox([left,right])

----------------

<font size=6> 4) Create Config Files

All customization and component settings are defined in the config files.

In [ ]:
#@title Set Sliders
#@markdown Set the appearance of the range slider elements and histograms.

try:
  import distinctipy
except:
  print("Installing distinctipy via Pip")
  !pip install distinctipy --quiet
  import distinctipy
  
# check if sliderCols exists
try:
  sliderCols
except NameError:
  sliderCols = list(sliderColumns.value)
  
if len(sliderCols) > 0:    
  layoutCol = {'width': '110px'}
  sliderColorDict = {}
  left = [Label('display name')]
  middle = [Label('description text')]
  right = [Label('histogram color')]
  colors = distinctipy.get_colors(len(sliderCols),pastel_factor=1)
  for i, sliderName in enumerate(sliderCols):
    sliderColorDict[sliderName] = widgets.ColorPicker(concise=False,value=distinctipy.get_hex(colors[i]),layout=layoutCol)
    right.append(sliderColorDict[sliderName])
  sliderInfoDict = {}
  for sliderName in sliderCols:
    sliderInfoDict[sliderName] = widgets.Text(placeholder="info text for slider",layout=layout)
    middle.append(sliderInfoDict[sliderName])
  sliderNameDict = {}
  for sliderName in sliderCols:
    sliderNameDict[sliderName] = widgets.Text(placeholder="name of slider",value=sliderName)
    left.append(sliderNameDict[sliderName])
  print("\nSlider Settings:\n") 
  idx = VBox([Label('')]+[Label(f"{n}:") for n in sliderCols])
  left_box = VBox([l for l in left])
  middle_box = VBox([m for m in middle])
  right_box = VBox([r for r in right])
  display(HBox([idx,left_box,middle_box,right_box]))
else:
  print("No Cluster fields selected!")

In [ ]:
#@title (optional) Set Cluster colors
#@markdown >Note: only necessary if categorical data was assigned for clusters

if len(classColumns.value) > 0:
  classColorDict = {}
  amount = len(classColumns.value)
  styleCol = {'description_width': '25px'}
  layoutCl = {'width': '135px'}
  allClasses = {}
  for className in classColumns.value:
    clusters = metadata[className].unique()
    allClasses[className] = len(clusters)
  l = sorted(allClasses.items(), key=lambda item: item[1])[0]
  length = max(allClasses.values())
  allColors = {}
  colors = distinctipy.get_colors(length)
  col = 5
  row = math.ceil(length/col)
  i=0
  rows = []
  for r in range(0,col):
    newRow = []
    for c in range(0,row):
      # classColorDict[className] = widgets.ColorPicker(concise=True, value=distinctipy.get_hex(colors[i]))
      if i < len(colors):
        allColors[i] = widgets.ColorPicker(concise=False, description=str(i), value=distinctipy.get_hex(colors[i]),layout=layoutCl,style=styleCol)
        newRow.append(allColors[i])
        i+=1
    rows.append(VBox([nr for nr in newRow]))
  display(HBox(rows))
else:
  allColors = False
  print("No cluster was selected.")

In [ ]:
#@title Create metadata.json
#@markdown Creates and saves the metadata.json file
#@markdown >Note: This step is necessary!

from CSN_utils import Utils

try:
  sliderCols
except NameError:
  sliderCols = list(sliderColumns.value)

imageFolder = f'public/datasets/{foldername}/images/'
if useExample.value == True:
  metadata["URL"] = metadata[filenameColumn.value]
else:
  metadata["URL"] = f"{imageFolder}/{metadata[filenameColumn.value]}"
metadataColumns = set(list(infoColumns.value) + sliderCols + list(filterColumns.value) + list(classColumns.value))
metadataColumns.add(filenameColumn.value)
metadata = metadata[metadataColumns]
Utils.write_metadata(foldername, metadata)

In [ ]:
#@title Calculate Histograms and create config files
#@markdown The CSN features Range Sliders with interactive histograms. This step calculates the necessary bins and prepares the data to display the histograms.

try:
    sliderCols
except NameError:
    sliderCols = list(sliderColumns.value)


from CSN_utils import HistogramGenerator, Utils

BarChartData = HistogramGenerator(foldername, data=metadata, selection=sliderCols, bucketCount = 50).generate()

def save_datasetsJSON():
  with open(f'build/datasets/datasets_config.json', "w") as fd:
    json.dump(datasetsJSON , fd)
  print("saved datasets_config.json")

def make_default(DEFAULT):
  datasetsJSON["default"] = DEFAULT
  print(f"changed default dataset to {datasetsJSON['data'][DEFAULT]['name']}")
  save_datasetsJSON()
  

sliderSetting = []

for k in sliderCols:
  dtype = 'float'
  if pd.api.types.is_integer_dtype(metadata[k]):
    dtype = 'int'
  slider = {"id":k,"title":sliderNameDict[k].value,"info":sliderInfoDict[k].value,"typeNumber":dtype,"color":sliderColorDict[k].value}
  sliderSetting.append(slider)
searchFields = []
for k in filterColumns.value:
  filter = {"columnField":k,"type":"selection"}
  searchFields.append(filter)
try:
    allColors
except NameError:
    clusters = {"clusterList":[],"clusterColors":[]}
else:
    clusters = {"clusterList":list(classColumns.value),"clusterColors":[allColors[g].value for g in allColors]}

    
configData = Utils.write_config(directory=foldername, title=datasetTitle.value, description=description.value, mappings=mappings, clusters=clusters, total=len(metadata), sliderSetting=sliderSetting, infoColumns=infoColumns.value, searchFields=searchFields, imageWebLocation=imageWebLocation.value, spriteRows=32, squareSize=2048, spriteSize=64, spriteDir=None)
newDataset = {'name': datasetTitle.value, 'directory': foldername}

datasetsJSON = {"default": 0, "data": [newDataset]}
save_datasetsJSON()

----------------

<font size=6> 5) Test and use your custom Collection Space Navigator

In [ ]:
#@title (optional) Download
#@markdown >Note: Download your CSN version 'CSN_build.zip' to use it locally. Only needed in Google Colab.

from google.colab import files

!7z a CSN_build.zip build

files.download('/content/CSN/CSN_build.zip')


In [ ]:
#@title (optional) Run proxy server in Google Colab (for quick testing)
#@markdown >Note: **This will run in Colab until stopped!** Sharing the link won't work and it might be a bit slow. Connection will close after few minutes.

try:
  from flask import Flask, render_template,send_from_directory
except:
  print("Installing flask via Pip")
  !pip install flask
  from flask import Flask, render_template,send_from_directory
  

import portpicker
import threading
import socket
import http.server
import socketserver
from functools import partial
from google.colab import output
from google.colab.output import eval_js

def server_entry():
    Handler = partial(http.server.SimpleHTTPRequestHandler, directory='/content/CSN/build')              
    httpd = socketserver.TCPServer(("", port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

port = portpicker.pick_unused_port()
thread = threading.Thread(target=server_entry)
thread.start()
output.serve_kernel_port_as_window(port)
port = portpicker.pick_unused_port()
proxy_URL = eval_js("google.colab.kernel.proxyPort(%d)" %port)

print("")
print(20*'#')
print("")
print(f"Use this url: {proxy_URL}")
print("")
print(20*'#')
print("")
print('starting server...')
print('Note: this will run in Colab until stopped!')


app = Flask(__name__,static_folder='/content/CSN/build/static',template_folder='/content/CSN/build')

@app.route('/<path:path>')
def send_report(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/', str(path))

@app.route('/CSN/static/<path:path>')
def send_report2(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/static/', str(path))

@app.route('/CSN/datasets/<path:path>')
def send_report3(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/datasets/', str(path))


@app.route("/")
def home():
    return render_template('index.html')
    
if __name__ == "__main__":
  app.run(debug=False, port=port)


In [ ]:
#@title (optional) Run ngrok server
ngrok_Authtoken = 'YOUR_NGROK_TOKEN' #@param {type:"string"}
#@markdown >Note: **This will run until stopped!** Requieres an ngrok account. See: https://ngrok.com/

try:
  from pyngrok import ngrok
  from flask_ngrok import run_with_ngrok
  from flask import Flask, render_template,send_from_directory
except:
  print("Installing flask, flask_ngrok and pyngrok via Pip")
  !pip install flask flask_ngrok pyngrok
  from flask import Flask, render_template,send_from_directory
  from pyngrok import ngrok
  from flask_ngrok import run_with_ngrok
  
app = Flask(__name__,static_folder='/content/CSN/build/',template_folder='/content/CSN/build/')

ngrok.set_auth_token(ngrok_Authtoken)
run_with_ngrok(app)
@app.route('/<path:path>')
def send_report(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/', str(path))

@app.route('/CSN/static/<path:path>')
def send_report2(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/static/', str(path))
  
@app.route('/CSN/datasets/<path:path>')
def send_report3(path):
  # remove the replace in next to lines later later <-- important !!!!!!!!
  print("files",path)
  return send_from_directory('/content/CSN/build/datasets/', str(path))

@app.route("/")
def home():
    return render_template('index.html')
    
if __name__ == "__main__":
  app.run()

## (optional) Run in localhost

>Note: only works locally (not within Colab)

To run your CSN version on localhost, unzip your downloaded file, open a terminal, navigate to your CSN directory and run `serve -s build`

The CSN should be then accessible at http://localhost:3000 in your browser.



### (optional) Use as production web tool

We recommend to use GitHub for hosting your custom CSN version and an external server for hosting your image collection. Note that GitHub limits any dataset to 1000 files.

To deploy your version as a web tool in GitHub:

>Note: Make sure your GitHub branch is called `gh-pages` and has the GitHub Pages option set. See more about GitHub Pages here: https://pages.github.com/

1. clone the official CSN repository: https://github.com/Collection-Space-Navigator/CSN  

2. install NVM: https://github.com/nvm-sh/nvm    

`curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.39.3/install.sh | bash
export NVM_DIR="$([ -z "${XDG_CONFIG_HOME-}" ] && printf %s "${HOME}/.nvm" || printf %s "${XDG_CONFIG_HOME}/nvm")"
[ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh" # This loads nvm`  

3. install node 16.16.0: by running `nvm install v16.16.0`  

4. replace the `CSN/build` folder with your own  

5. in `package.json,` change `"homepage": ""` to your GitHub pages URL (e.g. `"homepage": "https://collection-space-navigator.github.io/CSN"`)  

6. deploy the build folder to your GitHub pages by running `npm run deploy`  


For more information and other deployment options, see https://create-react-app.dev/docs/deployment/